# 🚀 CAUSAL AI - Quick Start

Ten notebook przeprowadzi Cię przez cały pipeline w 4 krokach.

---

## ETAP 1: Causal Discovery
Odkrywamy strukturę przyczynową z danych

In [1]:
import pandas as pd
import json
import numpy as np

from causal_discovery_engine import discover_causal_graph

# Załaduj dane
df = pd.read_csv("fraud_data.csv")
print(f"✓ Dane załadowane: {len(df)} wierszy, {len(df.columns)} kolumn")
print(f"  Kolumny: {list(df.columns)}")

✓ Dane załadowane: 50000 wierszy, 9 kolumn
  Kolumny: ['customer_age', 'customer_income', 'account_age_days', 'is_foreign_transaction', 'device_fingerprint_age_days', 'merchant_risk_score', 'transaction_amount', 'transaction_velocity_24h', 'is_fraud']


In [2]:
# Opcjonalnie: załaduj ground truth do walidacji
with open("synthetic_data/ground_truth_metadata.json") as f:
    gt = json.load(f)

gt_matrix = np.array(gt["ground_truth"]["adjacency_matrix"])
gt_variables = gt["ground_truth"]["variable_order"]

print(f"✓ Ground truth załadowany: {len(gt_variables)} zmiennych")

✓ Ground truth załadowany: 9 zmiennych


In [7]:
# Uruchom discovery (wybierz algorytm: 'lingam', 'pc', lub 'ges')
result = discover_causal_graph(
    data=df,
    algorithm="pc",
    ground_truth=gt_matrix,
    ground_truth_variables=gt_variables,
    print_report=True,
)

print(f"\n✓ Etap 1: Discovery complete")
print(f"  Odkryte krawędzie: {len(result.edges)}")

  0%|          | 0/9 [00:00<?, ?it/s]


                           CAUSAL DISCOVERY REPORT                            

Execution Summary
-----------------
  Algorithm:        PC
  Samples:          50,000
  Variables:        9
  Execution time:   0.483s
  Timestamp:        2025-12-10T17:53:28.362320+00:00

Discovered Causal Edges
-----------------------
  Source                    Target                      Strength     Conf
  ----------------------------------------------------------------------
  account_age_days          is_fraud                     +1.0000     1.00
  is_foreign_transaction    is_fraud                     +1.0000     1.00
  device_fingerprint_age_days is_fraud                     +1.0000     1.00
  merchant_risk_score       is_fraud                     +1.0000     1.00
  transaction_amount        is_fraud                     +1.0000     1.00
  transaction_velocity_24h  is_fraud                     +1.0000     1.00
  customer_income           transaction_amount           +0.9881     0.99
  transaction_a

In [8]:
# Zapisz wynik do JSON
discovery_output = {
    "algorithm": result.algorithm,
    "discovered_graph": {
        "edges": [
            {
                "source": e.source,
                "target": e.target,
                "strength": float(e.strength) if e.strength else 1.0,
            }
            for e in result.edges
        ],
        "variables": result.variable_names,
    },
    "metrics": {
        "precision": result.metrics.precision if result.metrics else None,
        "recall": result.metrics.recall if result.metrics else None,
        "f1": result.metrics.f1_score if result.metrics else None,
    }
}

with open("discovery_result.json", "w") as f:
    json.dump(discovery_output, f, indent=2)

print("✓ Zapisano: discovery_result.json")

✓ Zapisano: discovery_result.json


## ETAP 2: Human-in-the-Loop Review
Zatwierdzamy odkryte relacje

In [9]:
from causal_graph_review_svg import review_causal_graph

reviewer = review_causal_graph(
    discovery_path="discovery_result.json",
    ground_truth_path="synthetic_data/ground_truth_metadata.json",
)

# Interfejs pojawi się poniżej
# Kliknij "Auto-approve" lub zatwierdź ręcznie każdą krawędź

In [10]:
# WAŻNE: Uruchom tę komórkę PO zatwierdzeniu w UI powyżej
reviewer.save_approved("approved_graph.json")
print("✓ Etap 2: Review complete")

✓ Saved: approved_graph.json
  Approved: 7, Rejected: 1
✓ Etap 2: Review complete


### (Alternatywa) Auto-approve bez UI

In [ ]:
# Jeśli nie chcesz używać UI - odkomentuj i uruchom:

# from causal_graph_review_svg import review_causal_graph, EdgeStatus
# reviewer = review_causal_graph("discovery_result.json", "synthetic_data/ground_truth_metadata.json")
# for edge in reviewer.edges.values():
#     edge.status = EdgeStatus.APPROVED
#     edge.approved_strength = edge.ground_truth or edge.discovered_strength
# reviewer.save_approved("approved_graph.json")
# print("✓ Auto-approved")

## ETAP 3: Causal Effect Estimation (ATE/CATE)
Obliczamy siłę efektów przyczynowych

In [11]:
from causal_effect_estimator import CausalEffectEstimator

estimator = CausalEffectEstimator.from_files(
    approved_graph_path="approved_graph.json",
    data_path="fraud_data.csv",
    outcome_variable="is_fraud",
)

report = estimator.estimate_all()
report.display()

In [12]:
# Zapisz wyniki
report.to_json("causal_effects_report.json")
report.to_html("causal_effects_report.html")
print("✓ Etap 3: Effects complete")

✓ Report saved: causal_effects_report.json
✓ HTML report saved: causal_effects_report.html
✓ Etap 3: Effects complete


## ETAP 4: Counterfactual Reasoning
"Co by było gdyby?" dla pojedynczych transakcji

In [13]:
from counterfactual_engine import analyze_transaction

# Zdefiniuj transakcję do analizy
transaction = {
    "transaction_amount": 15000,
    "merchant_risk_score": 0.75,
    "transaction_velocity_24h": 6,
    "account_age_days": 60,
    "is_foreign_transaction": 1,
    "device_fingerprint_age_days": 10,
}

# Analiza
result = analyze_transaction(transaction)
print("✓ Etap 4: Counterfactual complete")

Czynnik,Aktualna → Percentyl,Zmiana P(fraud),Nowe P(fraud)
transaction_amount,15000.00 → 1622.16 (p25/p75),↓ -37.5pp,43.3%
transaction_velocity_24h,6.00 → 1.00 (p25/p75),↓ -11.2pp,69.6%
merchant_risk_score,0.75 → 0.16 (p25/p75),↓ -6.6pp,74.1%
is_foreign_transaction,1.00 → 0.00 (p25/p75),↓ -5.2pp,75.5%
account_age_days,60.00 → 505.00 (p25/p75),↓ -2.7pp,78.1%
device_fingerprint_age_days,10.00 → 124.00 (p25/p75),↓ -1.7pp,79.1%


✓ Etap 4: Counterfactual complete


---
## 🎉 GOTOWE!

Masz teraz:
- `discovery_result.json` - odkryty graf
- `approved_graph.json` - zatwierdzony graf
- `causal_effects_report.json` - wyniki ATE/CATE
- Analizę counterfactual dla wybranej transakcji

---

## ➡️ Następne kroki

1. **Walidacja:** Uruchom `validation.ipynb` żeby porównać discovery z ground truth
2. **Eksperymentuj:** Zmień algorytm na 'pc' lub 'ges' i porównaj wyniki
3. **Analizuj:** Sprawdź różne transakcje w counterfactual engine